# Colab-BasicSR

victorca25's BasicSR fork: [victorca25/BasicSR](https://github.com/victorca25/BasicSR)

Original colab by [nmkd](https://github.com/n00mkrad) with modifications by [styler00dollar](https://github.com/styler00dollar)

In [ ]:
#@title Check GPU

gpu = !nvidia-smi --query-gpu=gpu_name --format=csv
print("GPU: " + gpu[1])

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install

In [ ]:
#@title Install

!rm -r "/content/BasicSR"
!mkdir "/content/BasicSR"

from datetime import datetime, timedelta
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
import os
import fileinput
import sys

sedloc = ""
%cd /content/
#Install apt-fast, for faster installing
!/bin/bash -c "$(curl -sL https://git.io/vokNn)"
#Get some basic dependencies
!apt-fast install -y -q -q p7zip-full p7zip-rar

dr_exists = os.path.isdir("BasicSR/")

!pip install opencv-python
!pip install tensorboardX
!pip install pyyaml

# Clone BasicSR (vic's dev2 branch)
!rm -r /content/BasicSR
%cd /content/
#!git clone "https://github.com/victorca25/BasicSR.git"
!git clone https://github.com/styler00dollar/Colab-BasicSR BasicSR
%cd /content/BasicSR
#!git checkout dev2

# create empty folders
!mkdir /content/hr
!mkdir /content/lr
!mkdir /content/lr_val
!mkdir /content/hr_val

print('Done.')

# Dataset

Pre-configured paths:

LR: ```/content/lr```

HR: ```/content/hr```

LR_VAL: ```/content/lr_val```

HR_VAL: ```/content/hr_val```

# Download data

You need to upload the data and then extract it within colab. You can use Google Drive for that.

In [ ]:
!cp "/content/drive/My Drive/data.7z" "/content/data.7z"
!7z x /content/data.7z

In [ ]:
# Download pretrain models if needed (Alternatively, you can load them from Google Drive)
#!wget "http://dl.nmkd.de/ai/esrgan/1xESRGAN.pth" --content-disposition -P /content/BasicSR/experiments/pretrained_models/
#!wget "http://dl.nmkd.de/ai/esrgan/4xESRGAN.pth" --content-disposition -P /content/BasicSR/experiments/pretrained_models/
#!wget "http://dl.nmkd.de/ai/esrgan/8xESRGAN.pth" --content-disposition -P /content/BasicSR/experiments/pretrained_models/

In [ ]:
#@title Check amount of files in paths
"""
%cd /content/hr
!ls | wc -l
%cd /content/lr
!ls | wc -l
"""

In [ ]:
#@title example config
%%writefile /content/BasicSR/codes/options/train/train_template.yml
name: debug_001_template # use "debug" or "debug_nochkp" in the name to run a test session and check everything is working. Does validation and state saving every 8 iterations.
#name: 001_template # remove "debug" to run the real training session.
use_tb_logger: true
model: AdaFill #srragan | sr | srgan | ppon | asrragan | DFNet | EdgeConnect | CSA | RN | deepfillv1 | deepfillv2 | Adaptive | Global | Pluralistic | crfill | DeepDFNet | partial | DMFN | pennet | LBAM | RFR | FRRN | PRVS | CRA | USRNet | atrous | MEDFE | AdaFill
scale: 1
gpu_ids: [0]
use_amp: false
use_swa: false

# Dataset options:
datasets:
  train:
    name: DIV2K
    mode: LRHRC # LR | LRHR | LRHROTF | LRHRC | LRHRseg_bg |VLRHR | VLR | LRHRPBR
    # dataroot_HR: '../datasets/train/hr' # Original, with a single directory
    # dataroot_LR: '../datasets/train/lr' # Original, with a single directory
    dataroot_HR: [
      '/content/hr'
      ] # high resolution / ground truth images
    dataroot_LR: [
      #'/content/lr' #,
      # '../datasets/train/lr3'
      ] # low resolution images. If there are missing LR images, they will be generated on the fly from HR
    subset_file: null
    use_shuffle: true
    znorm: false # true | false # To normalize images in [-1, 1] range. Default = None (range [0,1]). Can use with activation function like tanh.
    n_workers: 1 # 0 to disable CPU multithreading, or an integrer representing CPU threads to use for dataloading
    batch_size: 2
    virtual_batch_size: 1
    HR_size: 256 # patch size. Default: 128. Needs to be coordinated with the patch size of the features network
    image_channels: 3 # number of channels to load images in

    # Color space conversion: 'color' for both LR and HR, 'color_LR' for LR independently, 'color_HR' for HR independently
    # color: 'y' # remove for no conversion (RGB) | 'y' for Y in YCbCr | 'gray' to convert RGB to grayscale | 'RGB' to convert gray to RGB
    # color_LR: 'y' # remove for no conversion (RGB) | 'y' for Y in YCbCr | 'gray' to convert RGB to grayscale | 'RGB' to convert gray to RGB
    # color_HR: 'y' # remove for no conversion (RGB) | 'y' for Y in YCbCr | 'gray' to convert RGB to grayscale | 'RGB' to convert gray to RGB

    # LR and HR modifiers. Random flip LR and HR or ignore provided LRs and generate new ones on the fly with defined probability:
    # rand_flip_LR_HR: false # true # flip LR and HR during training.
    # flip_chance: 0.05 # Example: 0.05 = 5% chance of LR and HR flipping during training.
    # aug_downscale: 0.2 # Example: 0.6 = 60% chance of generating LR on the fly, even if LR dataset exists

    # If manually configuring on the fly generation of LR: (else, it will automatically default to Matlab-like downscale algorithm (777) when/if required
    #dataroot_kernels: '/content/kernels_4x/kernelGAN/' # location of the image kernels extracted with KernelGAN, for use with the "realistic" downscale type
    lr_downscale: True # false
    lr_downscale_types: ["linear", "cubic", "matlab_bicubic"] # select from ["nearest", "linear", "cubic", "area", "lanczos4", "linear_exact", "matlab_bicubic", "realistic"] #scale_algos #scaling interpolation options

    # Rotations augmentations:
    use_flip: true # flip images
    use_rot: true # rotate images in 90 degree angles
    hr_rrot: false # rotate images in random degress between -45 and 45

    # Noise and blur augmentations:
    lr_blur: false # true | false
    lr_blur_types: {gaussian: 1, clean: 3} # select from: "average","box","gaussian","bilateral","clean" ##blur options #median and motion aren't working yet
    noise_data: ../noise_patches/normal/ # location of the noise patches extracted from real images to use for noise injection with noise option "patches"
    lr_noise: false # true | false
    lr_noise_types: {gaussian: 1, JPEG: 1, clean: 4} # select from: "gaussian", "JPEG", "quantize", "poisson", "dither", "s&p", "speckle", "patches", "clean"
    lr_noise2: false # true | false
    lr_noise_types2: {dither: 2, clean: 2} # select from: "gaussian", "JPEG", "quantize", "poisson", "dither", "s&p", "speckle", "patches", "clean"
    hr_noise: false # true | false
    hr_noise_types:  {gaussian: 1, clean: 4} # select from: "gaussian", "JPEG", "quantize", "poisson", "dither", "s&p", "speckle", "clean"

    # Color augmentations
    # lr_fringes: true # true | false
    # lr_fringes_chance: 0.4
    # auto_levels: HR # "HR" | "LR" | "Both" #add auto levels to the images to expand dynamic range. Can use with SPL loss or (MS)-SSIM.
    # rand_auto_levels: 0.7 # Example: 0.4 = 40% chance of adding auto levels to images on the fly
    # lr_unsharp_mask: true # add a unsharpening mask to LR images. Can work well together with the HFEN loss function.
    # lr_rand_unsharp: 1 # Example: 0.5 = 50% chance of adding unsharpening mask to LR images on the fly
    # hr_unsharp_mask: true # add a unsharpening mask to HR images. Can work well together with the HFEN loss function.
    # hr_rand_unsharp: 1 # Example: 0.5 = 50% chance of adding unsharpening mask to HR images on the fly

    # Augmentations for classification or (maybe) inpainting networks:
    # lr_cutout: false # true | false
    # lr_erasing: false # true | false

    # Options for inpainting (Options currently only for 1x inpainting in mind.)
    center_crop: False # Will centergrop LR and HR images and keep aspect ratio
    resize_HR_dimension: False # Downscale lr and hr to given HR value (if you don't set this to true and the image is bigger than needed, random crop will be applied. Should be used with center crop)

    training_with_canny: False # ONLY needed for EdgeConnect and PRVS
    training_with_canny_SR: False # ONLY needed for sisr
    noise_estimation: False # ONLY enable this for USRNet

  val:
    name: val_set14_part
    mode: LRHROTF
    dataroot_HR: '/content/hr_val'
    #dataroot_LR: '/content/lr_val'

    znorm: false # true | false # To normalize images in [-1, 1] range. Default = None (range [0,1]). Can use with activation function like tanh.

    # Color space conversion: 'color' for both LR and HR, 'color_LR' for LR independently, 'color_HR' for HR independently
    # color: 'y' # remove for no conversion (RGB) | 'y' for Y in YCbCr | 'gray' to convert RGB to grayscale | 'RGB' to convert gray to RGB
    # color_LR: 'y' # remove for no conversion (RGB) | 'y' for Y in YCbCr | 'gray' to convert RGB to grayscale | 'RGB' to convert gray to RGB
    # color_HR: 'y' # remove for no conversion (RGB) | 'y' for Y in YCbCr | 'gray' to convert RGB to grayscale | 'RGB' to convert gray to RGB

    # hr_crop: false #disabled
    lr_downscale: True # Needs to be true if you don't train 1x

    #dataroot_kernels: '/content/kernels_4x/kernelGAN/' # if kernels exist, downscale will happen with them, if not, then lr_downscale_types will be applied
    lr_downscale_types: ["linear", "cubic"] # select from ["nearest", "linear", "cubic", "area", "lanczos4", "linear_exact", "matlab_bicubic"] #scale_algos #scaling interpolation options

    training_with_canny: False # ONLY needed for EdgeConnect and PRVS
    training_with_canny_SR: False # ONLY needed for sisr
    noise_estimation: False # ONLY enable this for USRNet

path:
    strict: false # true | false
    root: '/content/BasicSR/'
    #pretrain_model_G: '../experiments/pretrained_models/RRDB_PSNR_x4.pth'
    #pretrain_model_G: '../experiments/pretrained_models/RRDB_ESRGAN_x4.pth'
    #pretrain_model_G: '../experiments/pretrained_models/nESRGANplus.pth'
    #pretrain_model_G: '../experiments/pretrained_models/PPON_G.pth'
    # pretrain_model_G: '../experiments/pretrained_models/PANx4_DF2K.pth'
    #resume_state: '../experiments/debug_002_RRDB_ESRGAN_x4_DIV2K/training_state/16.state'

# Generator options:
network_G:
    # ESRGAN:
    #which_model_G: RRDB_net # RRDB_net (original ESRGAN arch) | MRRDB_net (modified/"new" arch) | sr_resnet
    #norm_type: null
    #mode: CNA
    #nf: 64 # of discrim filters in the first conv layer
    #nb: 23
    #in_nc: 3 # of input image channels: 3 for RGB and 1 for grayscale
    #out_nc: 3 # of output image channels: 3 for RGB and 1 for grayscale
    #gc: 32
    #group: 1
    #convtype: Conv2D # Conv2D | PartialConv2D
    #net_act: leakyrelu # swish | leakyrelu
    #gaussian: true # true | false
    #plus: false # true | false
    ##finalact: tanh # Test. Activation function to make outputs fit in [-1, 1] range. Default = None. Coordinate with znorm.

    # ASRGAN:
    #which_model_G: asr_resnet # asr_resnet | asr_cnn
    #nf: 64

    # PPON:
    #which_model_G: ppon # | ppon
    ##norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 24
    #in_nc: 3
    #out_nc: 3
    ##gc: 32
    #group: 1
    ##convtype: Conv2D #Conv2D | PartialConv2D

    # SRGAN:
    #which_model_G: sr_resnet # RRDB_net | sr_resnet
    #norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 16
    #in_nc: 3
    #out_nc: 3

    # SR:
    #which_model_G: RRDB_net # RRDB_net | sr_resnet
    #norm_type: null
    #mode: CNA
    #nf: 64
    #nb: 23
    #in_nc: 3
    #out_nc: 3
    #gc: 32
    #group: 1

    # PAN:
    # which_model_G: pan_net
    # in_nc: 3
    # out_nc: 3
    # nf: 40
    # unf: 24
    # nb: 16
    # self_attention: true
    # double_scpa: false

    # edge-informed-sisr
    #which_model_G: sisr
    #use_spectral_norm: True

    # USRNet
    #which_model_G: USRNet
    #in_nc=4
    #out_nc=3
    #nc=[64, 128, 256, 512]
    #nb=2
    #act_mode='R'
    #downsample_mode='strideconv'
    #upsample_mode='convtranspose'
    
    # ----Inpainting Generators----
    # DFNet (batch_size: 2+, needs 2^x image input and validation) (2019)
    #which_model_G: DFNet
    #c_img: 3
    #c_mask: 1
    #c_alpha: 3
    #mode: nearest
    #norm: batch
    #act_en: relu
    #act_de: leaky_relu
    #en_ksize: [7, 5, 5, 3, 3, 3, 3, 3]
    #de_ksize: [3, 3, 3, 3, 3, 3, 3, 3]
    #blend_layers: [0, 1, 2, 3, 4, 5]
    #conv_type: deform # partial | normal | deform

    # EdgeConnect (2019)
    #which_model_G: EdgeConnect
    #use_spectral_norm: True
    #residual_blocks_edge: 8
    #residual_blocks_inpaint: 8
    #conv_type_edge: 'normal' # normal | partial | deform (has no spectral_norm)
    #conv_type_inpaint: 'normal' # normal | partial | deform

    # CSA (2019)
    #which_model_G: CSA
    #c_img: 3
    #norm: 'instance'
    #act_en: 'leaky_relu'
    #act_de: 'relu'

    # RN (2020)
    #which_model_G: RN
    #input_channels: 3
    #residual_blocks: 8
    #threshold: 0.8

    # deepfillv1 (2018)
    #which_model_G:  deepfillv1

    # deepfillv2 (2019)
    #which_model_G: deepfillv2
    #in_channels:  4
    #out_channels:  3
    #latent_channels:  64
    #pad_type:  'zero'
    #activation:  'lrelu' 
    #norm: 'in'
    #conv_type: partial # partial | normal

    # Adaptive (2020)
    #which_model_G: Adaptive
    #in_channels: 3
    #residual_blocks: 1
    #init_weights: True

    # Global (2020)
    #which_model_G: Global
    #input_dim: 5
    #ngf: 32
    #use_cuda: True
    #device_ids: [0]

    # Pluralistic (2019)
    #which_model_G: Pluralistic
    #ngf_E: 32
    #z_nc_E: 128
    #img_f_E: 128
    #layers_E: 5
    #norm_E: 'none'
    #activation_E: 'LeakyReLU'
    #ngf_G: 32
    #z_nc_G: 128
    #img_f_G: 128
    #L_G: 0
    #output_scale_G: 1
    #norm_G: 'instance'
    #activation_G: 'LeakyReLU'

    # crfill (2020)
    #which_model_G: crfill
    #cnum: 48

    # DeepDFNet (experimental)
    #which_model_G: DeepDFNet
    #in_channels:  4
    #out_channels:  3
    #latent_channels:  64
    #pad_type:  'zero'
    #activation:  'lrelu' 
    #norm: 'in'

    # partial (2018)
    #which_model_G: partial

    # DMFN (2020)
    #which_model_G: DMFN
    #in_nc: 4
    #out_nc: 3
    #nf: 64
    #n_res: 8 
    #norm: 'in'
    #activation: 'relu'

    # pennet (2019)
    #which_model_G: pennet

    # LBAM (2019)
    #which_model_G: LBAM
    #inputChannels: 4
    #outputChannels: 3

    # RFR (use_swa: false) (2020)
    #which_model_G: RFR
    #conv_type: deform # partial | deform

    # FRRN (2019)
    #which_model_G: FRRN

    # PRVS (2019)
    #which_model_G: PRVS

    # CRA (HR_size: 512) (2020)
    #which_model_G: CRA
    #activation: 'elu'
    #norm: 'none'

    # atrous (2020)
    #which_model_G: atrous

    # MEDFE (batch_size: 1) (2020)
    #which_model_G: MEDFE

    # AdaFill (2021)
    which_model_G: AdaFill

# Discriminator options:
network_D:
    # ESRGAN (default)| PPON:
    which_model_D: context_encoder # discriminator_vgg_128 | discriminator_vgg | discriminator_vgg_128_fea (feature extraction) | discriminator_vgg_fea (feature extraction) | patchgan | multiscale | context_encoder
    norm_type: batch
    act_type: leakyrelu
    mode: CNA # CNA | NAC
    nf: 64
    in_nc: 3
    nlayer: 3 # only for patchgan and multiscale
    num_D: 3 # only for multiscale

# Schedulers options:
train:
    lr_G: 0.0001 #0.0001 # 2e-4 # starting lr_g #Test, default: 1e-4
    weight_decay_G: 0
    beta1_G: 0.9
    lr_D: 0.0001 #0.0001 # 2e-4 # starting lr_d #Test, default: 1e-4
    weight_decay_D: 0
    beta1_D: 0.9

    # For MultiStepLR (ESRGAN, default):
    lr_scheme: MultiStepLR
    lr_steps: [50000, 100000, 200000, 300000] # training from scratch
    #lr_steps: [50000, 75000, 85000, 100000] #finetuning
    lr_gamma: 0.5 # lr change at every step (multiplied by)

    # For StepLR_Restart (PPON):
    # lr_gamma: 0.9 #lr change at every step (multiplied by)
    # lr_scheme: StepLR_Restart # MultiStepLR | MultiStepLR_Restart | StepLR | StepLR_Restart | CosineAnnealingLR_Restart
    # lr_step_sizes: [200, 100, 250] # Steps for each restart for "StepLR_Restart"
    # restarts: [138000, 172500] # Restart iterations for "MultiStepLR_Restart", "StepLR_Restart" and "CosineAnnealingLR_Restart"
    # restart_weights: [1, 0.5, 0.5] #lr_() * each weight in "restart_weights" for each restart in "restarts"
    ##clear_state: true

    # For MultiStepLR_Restart:
    # lr_gamma: 0.9
    # lr_scheme: MultiStepLR_Restart # MultiStepLR | MultiStepLR_Restart | StepLR | StepLR_Restart | CosineAnnealingLR_Restart
    # lr_steps: [34500, 69000, 103500, 155250, 189750, 241500] #For "MultiStepLR" and "MultiStepLR_Restart"
    # restarts: [138000, 172500] # Restart iterations for "MultiStepLR_Restart", "StepLR_Restart" and "CosineAnnealingLR_Restart"
    # restart_weights: [0.5, 0.5] # lr_() * each weight in "restart_weights" for each restart in "restarts"
    ##clear_state: true

    # For CosineAnnealingLR_Restart (PAN)
    # lr_G: !!float 7e-4
    # lr_scheme: "CosineAnnealingLR_Restart"
    # beta1_G: 0.9
    # beta2_G: 0.99
    # lr_D: 7e-4
    # beta1_D: 0.9
    # beta2_D: 0.99
    # # beta1: 0.9
    # # beta2: 0.99
    # niter: 1000000
    # warmup_iter: -1  # no warm up
    # T_period: [250000, 250000, 250000, 250000]
    # restarts: [250000, 500000, 750000]
    # restart_weights: [1, 1, 1]
    # eta_min: !!float 1e-7

    # For SWA scheduler
    swa_start_iter: 375000 #Just reference: 75% of 500000. Can be any value, including 0 to start right away with a pretrained model.
    swa_lr: 1e-4 #Has to be ~order of magnitude of a stable lr for the regular scheduler
    swa_anneal_epochs: 10
    swa_anneal_strategy: "cos"

    # Losses:
    pixel_criterion: l1 # "l1" | "l2" | "cb" | "elastic" | "relativel1" | "l1cosinesim" | "clipl1" #pixel loss
    pixel_weight: 1e-2 # 1e-2 | 1
    feature_criterion: l1 # "l1" | "l2" | "cb" | "elastic" #feature loss (VGG feature network)
    feature_weight: 1
    # cx_weight: 0.5
    # cx_type: contextual
    # cx_vgg_layers: {conv_3_2: 1, conv_4_2: 1}
    # hfen_criterion: l1 # hfen: "l1" | "l2" | "rel_l1" | "rel_l2" #helps in deblurring and finding edges, lines
    # hfen_weight: 1e-6
    # grad_type: grad-4d-l1 # 2d | 4d / - any of the pixel crit, ie "grad-2d-l1"
    # grad_weight: 4e-1 # 4e-1
    # tv_type: 4D # "normal" | "4D" #helps in denoising, reducing upscale artefacts
    # tv_weight: 1e-5 # Change "tv_weight" so the l_g_tv is around 1e-02 - 1e-01
    # tv_norm: 1 # 1 for l1 (default) or 2 for l2.
    # ssim_type: ssim # "ssim" | "ms-ssim" #helps to maintain luminance, contrast and covariance between SR and HR
    # ssim_weight: 1
    # lpips_weight: 1 # perceptual loss
    # lpips_type: net-lin # net-lin | net *
    # lpips_net: squeeze # "vgg" | "alex" | "squeeze"

    # Experimental losses
    # spl_type: spl # "spl" | "gpl" | "cpl"
    # spl_weight: 0.1 # 1e-2 # SPL loss function. note: needs to add a cap in the generator (finalcap; For [0,1] range -> "finalcap": "clamp") or the overflow loss or it can become unstable.
    # of_type: overflow # overflow loss function to force the images back into the [0, 1] range
    # of_weight: 0.2
    # fft_type: fft
    # fft_weight: 0.1
    # color_criterion: color-l1cosinesim # l1cosinesim naturally helps color consistency, so it is the best to use here, but others can be used as well
    # color_weight: 1 # Loss based on the UV channels of YUV color space, helps preserve color consistency
    # avg_criterion: avg-l1
    # avg_weight: 5 # Averaging downscale loss
    # ms_criterion: multiscale-l1
    # ms_weight: 1e-2

    # Adversarial loss:
    gan_type: vanilla # "vanilla" | "wgan-gp" | "lsgan"
    gan_weight: 5e-3 # * test: 7e-3
    # for wgan-gp
    # D_update_ratio: 1
    # D_init_iters: 0
    # gp_weigth: 10
    # if using the discriminator_vgg_128_fea or discriminator_vgg_fea feature maps to calculate feature loss
    # gan_featmaps: true # true | false
    # dis_feature_criterion: cb # "l1" | "l2" | "cb" | "elastic" #discriminator feature loss
    # dis_feature_weight: 0.01 # 1

    # For PPON
    # train_phase: 1 # Training starting phase, can skip the first phases
    # phase1_s: 100 # 5000 #100 #5000 #138000 #-1 to skip. Need to coordinate with the LR steps. #COBranch: lr =  2e−4, decreased by the factor of 2 for every 1000 epochs (1.38e+5 iterations) 138k
    # phase2_s: 200 # 10000 #200 #10000 #172500 #-1 to skip. Need to coordinate with the LR steps. #SOBranch: λ = 1e+3 (?), lr = 1e−4 and halved at every 250 epochs (3.45e+4iterations) 34.5k
    # phase3_s: 5000000 # 207000 #-1 to skip. Need to coordinate with the LR steps. #POBranch: η = 5e−3,  lr = 1e−4 and halved at every 250 epochs (3.45e+4iterations) 34.5k
    # phase4_s: 100100

    # Differentiable Augmentation for Data-Efficient GAN Training
    # diffaug: true
    # dapolicy: 'color,transl_zoom,flip,rotate,cutout' # smart "all" (translation, zoom_in and zoom_out are exclusive)

    # Batch (Mixup) augmentations
    # mixup: true
    # mixopts: [blend, rgb, mixup, cutmix, cutmixup] # , "cutout", "cutblur"]
    # mixprob: [1.0, 1.0, 1.0, 1.0, 1.0] #, 1.0, 1.0]
    # mixalpha: [0.6, 1.0, 1.2, 0.7, 0.7] #, 0.001, 0.7]
    # aux_mixprob: 1.0
    # aux_mixalpha: 1.2
    ## mix_p: 1.2

    # Frequency Separator
    # fs: true
    # lpf_type: average # "average" | "gaussian"
    # hpf_type: average # "average" | "gaussian"

    # Other training options:
    # finalcap: clamp # Test. Cap Generator outputs to fit in: [-1, 1] range ("tanh"), rescale tanh to [0,1] range ("scaltanh"), cap ("sigmoid") or clamp ("clamp") to [0,1] range. Default = None. Coordinate with znorm. Required for SPL if using image range [0,1]
    manual_seed: 0
    niter: 5e5
    val_freq: 100 # 5e3
    # overwrite_val_imgs: true
    val_comparison: true
    metrics: 'psnr,ssim,lpips' # select from: "psnr,ssim,lpips" or a combination separated by comma ","

logger:
    print_freq: 25 #200
    save_checkpoint_freq: 5e3
    overwrite_chkp: false


# Train

In [ ]:
%cd '/content/BasicSR/codes'
# for super resolution
#!python train.py -opt "/content/BasicSR/codes/options/train/train_template.yml"
# for inpainting
!python train_inpaint.py -opt "/content/BasicSR/codes/options/train/train_template.yml"

In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/path/'

# Test

In [ ]:
# testing
%cd /content/BasicSR/codes/
!python test_inpaint.py --input /content/0.jpg --mask /content/mask.png --output /content/output.png --yaml "/content/BasicSR/codes/options/train/train_template.yml" \
                         --model_path "/content/BasicSR/experiments/001_template/models/latest_G.pth"